In [ ]:
# this is a example data set where i took few rows and column from the real data set to run since the real data set 
# was very large and took more time to process

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import numpy as np

# Load data
df = pd.read_csv("Book1.csv")

# Parse datetime
df['Current Date Time'] = pd.to_datetime(df['Current Date Time'])

# Optional: filter out invalid GPS or signal data
df = df[(df['Latitude'].notnull()) & (df['Longitude'].notnull())]
df = df[(df['RSRP LTE'].notnull()) & (df['CI LTE'].notnull())]

# Create a features dataframe for user fingerprinting
features = df[['Latitude', 'Longitude', 'RSRP LTE', 'CI LTE', 'TAC LTE', 'PCI LTE', 'EARFCN', 'Chip Operator']]

# Normalize the features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# DBSCAN clustering to estimate users
dbscan = DBSCAN(eps=0.5, min_samples=10)  # Tune eps if needed
clusters = dbscan.fit_predict(features_scaled)

# Assign cluster labels back to the DataFrame
df['user_cluster'] = clusters

# Count unique users (excluding noise labeled as -1)
num_users = df[df['user_cluster'] != -1]['user_cluster'].nunique()
print(f"Estimated number of unique users: {num_users}")


Estimated number of unique users: 2


C:\Users\karth\AppData\Local\Temp\ipykernel_2700\1188373181.py:10: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Current Date Time'] = pd.to_datetime(df['Current Date Time'])


In [4]:
# Sort data by time
df = df.sort_values(['user_cluster', 'Current Date Time'])

# Detect transitions
df['prev_CI'] = df.groupby('user_cluster')['CI LTE'].shift(1)
df['cell_changed'] = df['CI LTE'] != df['prev_CI']

# Count handovers per user
handover_counts = df[df['cell_changed'] & (df['user_cluster'] != -1)].groupby('user_cluster').size()
print(f"Average handovers per user: {handover_counts.mean():.2f}")
print(f"Total number of cell transitions (all users): {handover_counts.sum()}")


Average handovers per user: 3.00
Total number of cell transitions (all users): 6


In [6]:
import pandas as pd
import folium
from folium.plugins import AntPath
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from IPython.display import display

# Load and clean data
df = pd.read_csv("Book1.csv")
df['Current Date Time'] = pd.to_datetime(df['Current Date Time'])
df = df.dropna(subset=['Latitude', 'Longitude', 'CI LTE', 'PCI LTE'])

# Step 1: Cluster users by location
location_features = df[['Latitude', 'Longitude']]
location_scaled = StandardScaler().fit_transform(location_features)
df['user_cluster'] = DBSCAN(eps=0.3, min_samples=10).fit_predict(location_scaled)

# Step 2: Get all valid users (clusters not labeled as -1)
valid_clusters = df[df['user_cluster'] != -1]['user_cluster'].unique()

# Step 3: Loop over each user and visualize
for user_id in valid_clusters[:3]:  # Limit to first 3 users for clarity — adjust as needed
    user_df = df[df['user_cluster'] == user_id].copy().sort_values('Current Date Time')
    
    # Detect handover types
    user_df['prev_CI'] = user_df['CI LTE'].shift(1)
    user_df['prev_PCI'] = user_df['PCI LTE'].shift(1)
    
    def classify_handover(row):
        if pd.isna(row['prev_CI']) or pd.isna(row['prev_PCI']):
            return None
        if row['CI LTE'] != row['prev_CI']:
            return "inter-cell"
        elif row['PCI LTE'] != row['prev_PCI']:
            return "intra-cell"
        else:
            return None

    user_df['handover_type'] = user_df.apply(classify_handover, axis=1)

    # Initialize map
    start_coords = [user_df.iloc[0]['Latitude'], user_df.iloc[0]['Longitude']]
    m = folium.Map(location=start_coords, zoom_start=13)
    
    # Movement path
    path_coords = list(zip(user_df['Latitude'], user_df['Longitude']))
    AntPath(path_coords, color='blue', weight=4, delay=800).add_to(m)

    # Handover markers
    for _, row in user_df[user_df['handover_type'].notnull()].iterrows():
        ho_type = row['handover_type']
        icon_color = 'orange' if ho_type == 'intra-cell' else 'red'
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=f"{ho_type} handover\n{row['Current Date Time']}",
            icon=folium.Icon(color=icon_color, icon='signal')
        ).add_to(m)

    # Start & End markers
    folium.Marker(
        location=start_coords,
        popup=f"User {user_id} Start",
        icon=folium.Icon(color='green', icon='play')
    ).add_to(m)

    end_coords = [user_df.iloc[-1]['Latitude'], user_df.iloc[-1]['Longitude']]
    folium.Marker(
        location=end_coords,
        popup="End",
        icon=folium.Icon(color='purple', icon='stop')
    ).add_to(m)

    # Display map
    print(f"Map for User {user_id}")
    display(m)


C:\Users\karth\AppData\Local\Temp\ipykernel_23428\655259091.py:10: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Current Date Time'] = pd.to_datetime(df['Current Date Time'])


Map for User 0


Map for User 1


In [1]:
import pandas as pd
import folium
from folium.plugins import AntPath #Animates movement 
from sklearn.preprocessing import StandardScaler#Normalizes GPS coordinates before clustering.
from sklearn.cluster import DBSCAN #clustering algorithm to find user groups based on location
from IPython.display import display

    # Load and clean data
df = pd.read_csv("Book1.csv")
df['Current Date Time'] = pd.to_datetime(df['Current Date Time'])
df = df.dropna(subset=['Latitude', 'Longitude', 'CI LTE', 'PCI LTE', 'RSRP LTE', 'CQI LTE', 'SNR LTE'])

    # Step 1: Cluster users based on location
location_scaled = StandardScaler().fit_transform(df[['Latitude', 'Longitude']])
    #(normalizing lat n long)Rescale the values so that they are centered around 0 and have a consistent scale.
df['user_cluster'] = DBSCAN(eps=0.3, min_samples=10).fit_predict(location_scaled)
    # include all nearby points that are 0.3 units away in normalized coordinate space, eps = epsilon

    # Step 2: Analyze handovers per user
valid_clusters = df[df['user_cluster'] != -1]['user_cluster'].unique()

for user_id in valid_clusters[:3]:# analyze the first 3 users only, for performance
    user_df = df[df['user_cluster'] == user_id].copy().sort_values('Current Date Time')
    #work only on this users data and sort by time to track movement

    # Detect handovers
    user_df['prev_CI'] = user_df['CI LTE'].shift(1)
    user_df['prev_PCI'] = user_df['PCI LTE'].shift(1)

    def classify_handover(row):
        if pd.isna(row['prev_CI']) or pd.isna(row['prev_PCI']):
            return None
        if row['CI LTE'] != row['prev_CI']:
            return "inter-cell"
        elif row['PCI LTE'] != row['prev_PCI']:
            return "intra-cell"
        return None

    user_df['handover_type'] = user_df.apply(classify_handover, axis=1)

    #  calcuation of  handover
    def infer_reason(row):
        rsrp = row['RSRP LTE']
        cqi = row['CQI LTE']
        snr = row['SNR LTE']
        reasons = []
        if rsrp < -110:
            reasons.append("Low RSRP")
        if cqi < 5:
            reasons.append("Low CQI")
        if snr < 0:
            reasons.append("Low SNR")
        return ', '.join(reasons) if reasons else "Normal signal fluctuation"

    user_df['handover_reason'] = user_df.apply(
        lambda row: infer_reason(row) if pd.notnull(row['handover_type']) else None, axis=1
    )

    # Summary printout
    ho_summary = user_df['handover_type'].value_counts()
    inter_count = ho_summary.get("inter-cell", 0)
    intra_count = ho_summary.get("intra-cell", 0)
    total = inter_count + intra_count

    print(f"\n User {user_id + 1}")
    print(f" Total Handovers: {total}")
    print(f" Inter-cell: {inter_count}")
    print(f" Intra-cell: {intra_count}")
    print(" Handover Diagnostics:")

    for _, row in user_df[user_df['handover_type'].notnull()].iterrows():
        lat, lon = row['Latitude'], row['Longitude']
        ho_type = row['handover_type']
        time = row['Current Date Time']
        reason = row['handover_reason']
        if ho_type == "inter-cell":
            explanation = f"CI LTE changed from {row['prev_CI']} to {row['CI LTE']}"
        else:
            explanation = f"PCI LTE changed from {row['prev_PCI']} to {row['PCI LTE']} (CI LTE unchanged)"
        print(f"• {ho_type.upper()} @ ({lat:.5f}, {lon:.5f}) at {time}")
        print(f"   ↪ {explanation}")
        print(f"   ⚠ Likely Reason: {reason}")

    # Step 3: Map Visualization
    start_coords = [user_df.iloc[0]['Latitude'], user_df.iloc[0]['Longitude']]
    m = folium.Map(location=start_coords, zoom_start=13)

    path_coords = list(zip(user_df['Latitude'], user_df['Longitude']))
    AntPath(path_coords, color='blue', weight=4, delay=800).add_to(m)#nimated path connecting all GPS points.

    # Add handover markers with reason in popup
    for _, row in user_df[user_df['handover_type'].notnull()].iterrows():
        ho_type = row['handover_type']
        icon_color = 'red' if ho_type == 'inter-cell' else 'orange'
        folium.Marker(
            location=[row['Latitude'], row['Longitude']],
            popup=(f"{ho_type.upper()} Handover<br>"
                   f"Time: {row['Current Date Time']}<br>"
                   f"CI: {row['CI LTE']} | PCI: {row['PCI LTE']}<br>"
                   f"Reason: {row['handover_reason']}"),
            icon=folium.Icon(color=icon_color, icon='signal')
        ).add_to(m)

    # Start and end
    folium.Marker(
        location=start_coords,
        popup=f"User {user_id} Start",
        icon=folium.Icon(color='green', icon='play')
    ).add_to(m)

    end_coords = [user_df.iloc[-1]['Latitude'], user_df.iloc[-1]['Longitude']]
    folium.Marker(
        location=end_coords,
        popup="End",
        icon=folium.Icon(color='purple', icon='stop')
    ).add_to(m)

    display(m)


C:\Users\karth\AppData\Local\Temp\ipykernel_31560\290744961.py:10: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Current Date Time'] = pd.to_datetime(df['Current Date Time'])



 User 1
 Total Handovers: 62
 Inter-cell: 61
 Intra-cell: 1
 Handover Diagnostics:
• INTER-CELL @ (-3.07992, -60.00220) at 2023-08-21 09:48:00
   ↪ CI LTE changed from 235544839.0 to 235780357
   ⚠ Likely Reason: Normal signal fluctuation
• INTER-CELL @ (-3.04825, -60.00580) at 2023-08-21 11:54:00
   ↪ CI LTE changed from 235780357.0 to 235544862
   ⚠ Likely Reason: Normal signal fluctuation
• INTER-CELL @ (-3.04213, -60.00620) at 2023-08-21 12:55:00
   ↪ CI LTE changed from 235544862.0 to 235544861
   ⚠ Likely Reason: Low SNR
• INTER-CELL @ (-3.08839, -59.96510) at 2023-08-21 17:58:00
   ↪ CI LTE changed from 235544861.0 to 236666642
   ⚠ Likely Reason: Normal signal fluctuation
• INTER-CELL @ (-3.07925, -60.00150) at 2023-08-21 22:51:00
   ↪ CI LTE changed from 236666642.0 to 235575071
   ⚠ Likely Reason: Low SNR
• INTER-CELL @ (-3.13487, -60.01650) at 2023-10-02 15:17:00
   ↪ CI LTE changed from 235575071.0 to 82310451
   ⚠ Likely Reason: Low SNR
• INTER-CELL @ (-3.04270, -60.00610


 User 2
 Total Handovers: 9
 Inter-cell: 9
 Intra-cell: 0
 Handover Diagnostics:
• INTER-CELL @ (-3.04255, -60007.00000) at 2023-10-07 01:00:00
   ↪ CI LTE changed from 235544839.0 to 235570694
   ⚠ Likely Reason: Low SNR
• INTER-CELL @ (-3.04260, -60007.00000) at 2023-10-07 01:18:00
   ↪ CI LTE changed from 235570694.0 to 235544836
   ⚠ Likely Reason: Normal signal fluctuation
• INTER-CELL @ (-3.04260, -60007.00000) at 2023-10-07 06:18:00
   ↪ CI LTE changed from 235544836.0 to 235544833
   ⚠ Likely Reason: Normal signal fluctuation
• INTER-CELL @ (-3.09519, -60051.00000) at 2023-11-30 00:52:00
   ↪ CI LTE changed from 235544833.0 to 82430229
   ⚠ Likely Reason: Low RSRP, Low SNR
• INTER-CELL @ (-3.04261, -60007.00000) at 2024-02-06 08:58:00
   ↪ CI LTE changed from 82430229.0 to 82319627
   ⚠ Likely Reason: Normal signal fluctuation
• INTER-CELL @ (-3.08834, -59965.00000) at 2024-02-06 10:32:00
   ↪ CI LTE changed from 82319627.0 to 82314764
   ⚠ Likely Reason: Normal signal fluctua